In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import mlrose_hiive as mlrose
from mlrose_hiive import simulated_annealing as sa, random_hill_climb as rhc, genetic_alg as ga, mimic
from mlrose_hiive import RHCRunner, SARunner, GARunner, MIMICRunner, NNGSRunner, ExpDecay, GeomDecay, FourPeaks, FlipFlop
from mlrose_hiive.algorithms import gradient_descent as gd
import numpy as np
import pandas as pd
import random
import json
from randomized_optimization import run_problem_config, queens_problem, get_diabetes_data

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()

# notebook is mainly for exploration, use python script to generate final results

RS = 199101440
np.random.seed(RS) # keep results consistent
random.seed(RS)

from sklearn.metrics import accuracy_score

In [ ]:
n = 4
#roblem = queens_problem(n)
#problem = mlrose.DiscreteOpt(length=n, fitness_fn=FourPeaks(), maximize=True, max_val=2)
problem = mlrose.DiscreteOpt(length=n, fitness_fn=FlipFlop(), maximize=True, max_val=2)
#sa(problem,schedule=GeomDecay(),max_attempts=1000,max_iters=1000,fevals=True, random_state=RS)
#rhc(problem,max_attempts=100,max_iters=1000,random_state=RS, restarts=5)
#ga(problem)
mimic(problem)

In [ ]:
with open("nnconfig.json", 'r') as fp:
    config = json.load(fp)

In [ ]:
#results = run_problem_config(config)
#rhc_run_stats, rhc_run_curves, sa_run_stats, sa_run_curves, ga_run_stats, ga_run_curves, mimic_run_stats, mimic_run_curves = results

In [ ]:
x_train, x_test, y_train, y_test = get_diabetes_data()
train_df = pd.DataFrame(x_train, y_train)

In [ ]:
train_df.head()

In [ ]:
shared_params = {
    'max_iters': [1000],                  
    'learning_rate_init': [0.0002],               
    'activation': [mlrose.relu],
    'hidden_layer_sizes': [[49]]
}

runner_params = {
    "x_train" : x_train,
    "y_train" : y_train,
    "x_test" : x_test,
    "y_test" : y_test,
    "experiment_name": "nn",
    "clip_max": 5,
    "max_attempts": 10,
    "n_jobs": 8,
    "seed": RS,
    "cv": 2
}

In [ ]:
# gradient dececent
it_list = config['gd']['iteration_list']
gd_nnr = NNGSRunner(algorithm=gd, grid_search_parameters=shared_params,iteration_list=it_list, **runner_params)

# randomized hill climbing
rhc_params = shared_params | {"restarts" : config['rhc']['restart_list']}
it_list = config['rhc']['iteration_list']
rhc_nnr = NNGSRunner(algorithm=rhc, grid_search_parameters=rhc_params,iteration_list=it_list, **runner_params)

# simulated annealing
temp_list = [GeomDecay(init_temp=t) for t in config['sa']['temperature_list']]
sa_params = shared_params | {"schedule" : temp_list}
it_list = config['sa']['iteration_list']
sa_nnr = NNGSRunner(algorithm=sa, grid_search_parameters=sa_params,iteration_list=it_list, **runner_params)

# genetic algorithms
ga_params = {"mutation_prob" : config['ga']['mutation_rates'], "pop_size": config['ga']['population_sizes']}
ga_params = shared_params | ga_params
it_list = config['ga']['iteration_list']
ga_nnr = NNGSRunner(algorithm=ga, grid_search_parameters=ga_params,iteration_list=it_list, **runner_params)

In [ ]:
run_stats, curves, cv_results, grid_search_cv = ga_nnr.run()
run_stats = run_stats[run_stats['Iteration'] != 0]
run_stats = run_stats.query("Fitness == Fitness.min()")
run_stats.reset_index(inplace=True, drop=True)

In [ ]:
curves.plot(title=f"{run_stats['algorithm'][0]} NN Fitness over Iterations", xlabel="Iterations", ylabel="Fitness", x="Iteration", y="Fitness")

In [ ]:
run_stats

In [ ]:
cv_results

In [ ]:
y_test_pred = grid_search_cv.predict(x_test)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
print(y_test_accuracy)